# Example of data translation using a bijective graph
The graph method creates a series of interlocking nodes that have transformations between themselves. There are a few basic rules:
1. Any node has an edge into and out of itself
2. Any path that results in the same end location creates the same node 

## This example is one of a rectangular table, a set of column modeled data. 
### This graph has the folowing nodes, and hence transformations between all of them
1. n1, 'Pandas Data Frame', in memory 
2. n2, 'AsciiDataTable', in memory
3. n3, 'HDF File', on disk
4. n4, 'XML Data Table',in memory
5. n5, 'Excel File', on disk
6. n6, 'HTML String',in memory
7. 'JSON File', on disk
8. 'JSON String',in memory
9. 'CSV File', on disk
10. 'Matlab File', on disk

In [2]:
from pyMeasure import *
import pandas
from scipy.io import savemat,loadmat

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [3]:
import re
import datetime
import numpy as np

def edge_1_to_2(in_string):
    return in_string.splitlines()
    
def edge_2_to_1(string_list):
    return string_list_collapse(string_list)

class Graph():
    def __init__(self,**options):
        """Initializes the graph. The first 2 nodes and two edges forming a bijection between them are required"""
        defaults={"graph_name":"Graph",
                  "node_names":['n1','n2'],
                  "node_descriptions":{'n1':"A plain string",
                                       'n2':"A list of strings with no \\n, created with string.splitlines()"},
                  "current_node":'n1',
                  "state":[1,0],
                  "data":"This is a test string\n it has to have multiple lines \n and many characters 34%6\n^",
                  "edge_2_to_1":edge_2_to_1,
                  "edge_1_to_2":edge_1_to_2
                 }
        self.options={}
        for key,value in defaults.iteritems():
            self.options[key]=value
        for key,value in options.iteritems():
            self.options[key]=value
        self.elements=['graph_name','node_names','node_descriptions','current_node','state','data']
        for element in self.elements:
            self.__dict__[element]=self.options[element]
        self.edges=[]
        self.edge_matrices=[]
        self.state_matrix=np.matrix(self.state).T
        # Add the first 2 edges, required to intialize the graph properly
        self.add_edge(self.node_names[0],self.node_names[1],self.options["edge_1_to_2"])
        self.add_edge(self.node_names[1],self.node_names[0],self.options["edge_2_to_1"])
        
    def set_state(self,node_name,node_data):
        """Sets the graph state to be the state specified by node_name, and node_data"""
        try:
            current_node_state_position=self.node_names.index(node_name)
            self.current_node=node_name
            self.data=node_data
            self.state=[0 for i in range(len(self.node_names))]
            self.state[current_node_state_position]=1
            self.state_matrix=np.matrix(self.state).T
        except:
            print("Could not set the state of graph: {0}".format(self.graph_name))
            raise
            
    def add_edge(self,begin_node=None,end_node=None,edge_function=None):
        """Adds an edge mapping one node to another, required input is begin_node (it's name)
        end_node, and the edge function"""
        # check to see if edge is defined if it is increment a number
        edge_match=re.compile("edge_{0}_{1}".format(begin_node,end_node))
        keys=self.__dict__.keys()
        #print keys
        iterator=0
        for key in keys:
            if re.match(edge_match,key):
                iterator+=1
        edge_name="edge_{0}_{1}_{2:0>3d}".format(begin_node,end_node,iterator)
        self.__dict__[edge_name]=edge_function
        self.edges.append(edge_name)
        edge_matrix=np.zeros((len(self.state),len(self.state)))
        begin_position=self.node_names.index(begin_node)
        end_position=self.node_names.index(end_node)
        edge_matrix[end_position][begin_position]=1
        edge_matrix=np.matrix(edge_matrix)
        self.edge_matrices.append(edge_matrix)
        
    def move_to(self,path):
        """Changes the state of the graph by moving along the path specified"""
        #print path
        for index,edge in enumerate(path):
            #print edge
            edge_pattern='edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)'
            match=re.match(edge_pattern,edge)
            begin_node=match.groupdict()['begin_node']
            end_node=match.groupdict()['end_node']
            print("moving {0} -> {1}".format(begin_node,end_node))
            #print self.data
            self.data=self.__dict__[edge](self.data)
            #print self.data
            self.current_node=match.groupdict()['end_node']
            self.state=[0 for i in range(len(self.node_names))]
            position=self.node_names.index(self.current_node)
            self.state[position]=1
            self.state_matrix=np.matrix(self.state).T
            #print self.state
            #print self.current_node
            
    def virtual_move_to(self,path):
        """virtual_move_to simulates moving but does not change the state of the graph"""
        #print path
        temp_state=self.state
        temp_data=self.data
        temp_current_node=self.current_node
        temp_node_names=self.node_names
        for index,edge in enumerate(path):
            #print edge
            edge_pattern='edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)'
            match=re.match(edge_pattern,edge)
            begin_node=match.groupdict()['begin_node']
            end_node=match.groupdict()['end_node']
            #print("moving {0} -> {1}".format(begin_node,end_node))
            #print self.data
            temp_data=self.__dict__[edge](temp_data)
            #print self.data
            temp_current_node=match.groupdict()['end_node']
            temp_state=[0 for i in range(len(temp_node_names))]
            position=temp_node_names.index(temp_current_node)
            temp_state[position]=1
            #print temp_state
            #print self.state
            #print self.current_node  
            
    def __str__(self):
        return str(self.data)
    
    def add_node(self,node_name,edge_into_node_begin,edge_into_node_function,edge_out_node_end,
                 edge_out_node_function):
        """Adds a node to the graph. Required input is node_name (a string with no spaces), 
        a reference to an entering node,the function mapping the entering node to the new node, 
        a reference to an exiting node and the function mapping the
        new node to the exiting node."""
        # first check if node into and out of node is good
        self.node_names.append(node_name)
        self.state.append(0)
        self.state_matrix=np.matrix(self.state).T
        for index,matrix in enumerate(self.edge_matrices):
            pad_row=np.zeros((1,len(matrix)))
            new_matrix=np.concatenate((matrix, pad_row), axis=0)
            pad_column=np.zeros((1,len(self.node_names)))
            new_matrix=np.concatenate((new_matrix, pad_column.T), axis=1)
            #print("New matrix is :\n{0}".format(new_matrix))
            self.edge_matrices[index]=new_matrix
        self.add_edge(begin_node=node_name,end_node=edge_out_node_end,edge_function=edge_out_node_function)
        self.add_edge(begin_node=edge_into_node_begin,end_node=node_name,edge_function=edge_into_node_function)
    
    def path_length(self,path,num_repeats=10):
        """Determines the length of a given path, currently the metric is based on the time to move to."""
        begin_time=datetime.datetime.now()
        #num_repeats=100
        for i in range(num_repeats):
            self.virtual_move_to(path)
        end_time=datetime.datetime.now()
        delta_t=end_time-begin_time
        path_length=delta_t.total_seconds()/float(num_repeats)
        if path_length ==0.0:
            print("Warning the path length is less than 1 microsecond," 
                  "make sure num_repeats is high enough to measure it.")
        return path_length
                
    def is_path_valid(self,path):
        """Returns True if the path is valid from the current node position or False otherwise"""
        null_state=[0 for i in range(len(self.node_names))]
        null_state_matrix=np.matrix(null_state).T
        new_state=np.matrix(self.state).T
        for index,edge in enumerate(path):
            #print index
            #print edge
            edge_position=self.edges.index(edge)
            move_matrix=self.edge_matrices[edge_position]
            #print move_matrix
            new_state=move_matrix*new_state
            if new_state.any()==null_state_matrix.any():
                #print new_state
                #print null_state_matrix
                return False
        return True
            
            
    def get_entering_nodes(self,node): 
        """Returns all nodes that have an edge that enter the specificed node"""
        enter_edge_pattern=re.compile('edge_(?P<begin_node>\w+)_{0}_(?P<iterator>\w+)'.format(node))
        enter_nodes=[]
        for index,edge in enumerate(self.edges):
            enter_match=re.match(enter_edge_pattern,edge)
            if enter_match:
                enter_node=enter_match.groupdict()['begin_node']
                enter_nodes.append(enter_node)
        return enter_nodes
    
    def get_entering_edges(self,node): 
        """Returns all edges that enter the specificed node"""
        enter_edge_pattern=re.compile('edge_(?P<begin_node>\w+)_{0}_(?P<iterator>\w+)'.format(node))
        enter_edges=[]
        for index,edge in enumerate(self.edges):
            if re.match(enter_edge_pattern,edge):
                enter_edges.append(edge)
        return enter_edges
    
    def get_exiting_edges(self,node):
        """Returns all edges that exit the specificed node"""
        exit_edge_pattern=re.compile('edge_{0}_(?P<end_node>\w+)_(?P<iterator>\w+)'.format(node))
        exit_edges=[]
        for index,edge in enumerate(self.edges):
            if re.match(exit_edge_pattern,edge):
                exit_edges.append(edge)
        return exit_edges 
    
    def get_exiting_nodes(self,node): 
        """Returns all nodes that have an edge leaving the specificed node"""
        exit_edge_pattern=re.compile('edge_{0}_(?P<end_node>\w+)_(?P<iterator>\w+)'.format(node))
        exit_nodes=[]
        for index,edge in enumerate(self.edges):
            exit_match=re.match(exit_edge_pattern,edge)
            if exit_match:
                exit_node=exit_match.groupdict()['end_node']
                exit_nodes.append(exit_node)
        return exit_nodes
    
    def get_path(self,first_node,last_node):
        """Returns the first path found between first node and last node"""
        edge_pattern=re.compile('edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)')
        exit_paths=self.get_exiting_edges(first_node)
        next_nodes=self.get_exiting_nodes(first_node)
        #be careful here using the wrong assignment statement breaks this function 
        possible_paths=[]
        for exit_path in exit_paths:
            possible_paths.append([exit_path])
        #print("{0} is {1}".format('possible_paths',possible_paths))
        for i in range(len(self.node_names)):
            for index,path in enumerate(possible_paths):
                last_edge=path[-1]
                match=re.match(edge_pattern,last_edge)
                begin_node=match.groupdict()['begin_node']
                end_node=match.groupdict()['end_node']
                #print next_node
                if end_node==last_node:
                    #print("The path found is {0}".format(path))
                    return path
                next_possible_paths=[]
                next_edges=self.get_exiting_edges(end_node)
                next_nodes=self.get_exiting_nodes(end_node)
                #print("{0} is {1}".format('next_edges',next_edges))
                for index,next_edge in enumerate(next_edges):
                    #be careful here using the wrong assignment statement breaks this function 
                    #next_path=path is a deal breaker!! 
                    next_path=[]
                    for edge in path:
                        next_path.append(edge)
                    #print("{0} is {1}".format('next_path',next_path))
                    #print("{0} is {1}".format('next_edge',next_edge))
                    #next_node=next_nodes[index]
                    #print next_node
                    next_match=re.match(edge_pattern,next_edge)
                    next_node=next_match.groupdict()["end_node"]
                    begin_node_next_edge=next_match.groupdict()["begin_node"]
                    #print("{0} is {1}".format('next_node',next_node))
                    #print("{0} is {1}".format('begin_node_next_edge',begin_node_next_edge))
                    
                    if next_node==last_node and begin_node_next_edge==end_node:
                        next_path.append(next_edge)
                        #print("The path found is {0}".format(next_path))
                        return next_path
                    elif begin_node_next_edge==end_node:
                        next_path.append(next_edge)
                        next_possible_paths.append(next_path)
                        #print("{0} is {1}".format('next_possible_paths',next_possible_paths))
                    else:
                        pass
                    #print("{0} is {1}".format('next_possible_paths',next_possible_paths))
                possible_paths=next_possible_paths
                #print("{0} is {1}".format('possible_paths',possible_paths))
    
    def move_to_node(self,node):
        """Moves from current_node to the specified node"""
        path=self.get_path(self.current_node,node)
        self.move_to(path)
            
    def check_closed_path(self):
        """Checks that data is not changed for the first closed path found. Returns True if data==data after
        moving around the closed path, False otherwise. Starting point is current_node """
        temp_data=self.data
        path=self.get_path(self.current_node,self.current_node)
        if self.is_path_valid(path):
            pass
        else:
            print("Path is not valid, graph definition is broken")
            raise
        out=temp_data==self.data
        out_list=[self.current_node,path,out]
        print("The assertion that the data remains unchanged,\n" 
              "for node {0} following path {1} is {2}".format(*out_list))
        return out 
    
    def is_graph_isomorphic(self):
        """Returns True if all nodes have closed paths that preserve the data, False otherwise"""
        out=True
        for node in self.node_names:
            self.move_to_node(node)
            if not self.check_closed_path:
                out=False
        return out
            
            


In [4]:
options={}
options["data"]=[[1,2,3],[4,5,6]]
options["column_names"]=['a','b','c']
table=AsciiDataTable(None,**options)
table=OnePortCalrepModel(os.path.join(TESTS_DIRECTORY,'700437.asc'))
table=SNP(os.path.join(TESTS_DIRECTORY,'Solution_0.s4p'))
data_frame=AsciiDataTable_to_DataFrame(table)

The self.metadata["Device_Id"] variable is 700437


In [19]:
graph_options={"graph_name":"Rectangular Graph",
                  "node_names":['n1','n2'],
                  "node_descriptions":["Pandas Data Frame","AsciiDataTable"],
                  "current_node":'n1',
                  "state":[1,0],
                  "data":data_frame,
                  "edge_2_to_1":AsciiDataTable_to_DataFrame,
                  "edge_1_to_2":DataFrame_to_AsciiDataTable
                 }
rect_graph=Graph(**graph_options)

In [61]:
rect_graph.is_graph_isomorphic()

moving n1 -> n2
moving n2 -> n1
moving n1 -> n2


True

In [6]:
rect_graph.move_to_node("n2")

moving n2 -> n1
moving n1 -> n2


In [7]:
print rect_graph.data

a,b,c
1,2,3
4,5,6


In [20]:
def edge_1_to_3(pandas_data_frame):
    pandas_data_frame.to_hdf("Test.hdf","table")
    return "Test.hdf"
def edge_3_to_1(hdf_file_name):
    df=pandas.read_hdf(hdf_file_name,"table")
    return df

rect_graph.add_node("n3","n1",edge_1_to_3,"n1",edge_3_to_1)
rect_graph.node_descriptions.append("HDF File")

def edge_4_to_2(xml_table):
    
    table=AsciiDataTable(None,
                         column_names=xml_table.attribute_names,
                         data=xml_table.data)
    return table

def edge_2_to_4(data_table):
    xml=AsciiDataTable_to_XMLDataTable(data_table)
    return xml

rect_graph.add_node("n4","n2",edge_2_to_4,"n2",edge_4_to_2)
rect_graph.node_descriptions.append("XML Data Table")

# Need to add XML File and Html File using save and save_HTML()


def edge_1_to_5(pandas_data_frame,file_name="Test.xlsx"):
    pandas_data_frame.to_excel(file_name,index=False)
    return file_name

def edge_5_to_1(excel_file_name):
    df=pandas.read_excel(excel_file_name)
    return df

rect_graph.add_node("n5","n1",edge_1_to_5,"n1",edge_5_to_1)
rect_graph.node_descriptions.append("Excel File")

def edge_1_to_6(pandas_data_frame):
    html=pandas_data_frame.to_html(index=False)
    return html

def edge_6_to_1(html_string):
    list_df=pandas.read_html(html_string)
    return list_df[0]

rect_graph.add_node("n6","n1",edge_1_to_6,"n1",edge_6_to_1)
rect_graph.node_descriptions.append("HTML String")

# Note a lot of the pandas reading and writing cause float64 round off errors
# applymap(lambda x: np.around(x,10) any all float fields will fix this
# also the column names move about in order

def edge_1_to_7(pandas_data_frame):
    json=pandas_data_frame.to_json("test.json",orient='records')
    return "test.json"

def edge_7_to_1(json_file_name):
    data_frame=pandas.read_json(json_file_name,orient='records')
    return data_frame

rect_graph.add_node("n7","n1",edge_1_to_7,"n1",edge_7_to_1)
rect_graph.node_descriptions.append("JSON File")

def edge_1_to_8(pandas_data_frame):
    json=pandas_data_frame.to_json(orient='records')
    return json

def edge_8_to_1(json_string):
    data_frame=pandas.read_json(json_string,orient='records')
    return data_frame

rect_graph.add_node("n8","n1",edge_1_to_8,"n1",edge_8_to_1)
rect_graph.node_descriptions.append("JSON String")

def edge_1_to_9(pandas_data_frame,file_name="test.csv"):
    json=pandas_data_frame.to_csv(file_name,index=False)
    return file_name

def edge_9_to_1(csv_file_name):
    data_frame=pandas.read_csv(csv_file_name)
    return data_frame

rect_graph.add_node("n9","n1",edge_1_to_9,"n1",edge_9_to_1)
rect_graph.node_descriptions.append("CSV File")

def edge_2_to_10(ascii_data_table,file_name="test.mat"):
    matlab_data_dictionary={"data":ascii_data_table.data,"column_names":ascii_data_table.column_names}
    savemat(file_name,matlab_data_dictionary)
    return file_name

def edge_10_to_2(matlab_file_name):
    matlab_data_dictionary=loadmat(matlab_file_name)
    ascii_data_table=AsciiDataTable(None,
                                    column_names=map(lambda x: x.rstrip().lstrip(),
                                                     data["column_names"].tolist()),
                                     data=data["data"].tolist())
    return ascii_data_table

rect_graph.add_node("n10","n2",edge_2_to_10,"n2",edge_10_to_2)
rect_graph.node_descriptions.append("Matlab File")

def edge_7_to_4(json_file_string):
    data_dictionary_list=json.load(open(json_file_string,'r'))
    xml=DataTable(None,data_dictionary={"data":data_dictionary_list})
    return xml
rect_graph.add_edge("n7","n4",edge_7_to_4)

In [21]:
rect_graph.is_graph_isomorphic()

moving n1 -> n2
moving n2 -> n1
moving n1 -> n2
moving n2 -> n1
moving n1 -> n3
moving n3 -> n1
moving n1 -> n2
moving n2 -> n4
moving n4 -> n2
moving n2 -> n1
moving n1 -> n5
moving n5 -> n1
moving n1 -> n6
moving n6 -> n1
moving n1 -> n7
moving n7 -> n1
moving n1 -> n8
moving n8 -> n1
moving n1 -> n9
moving n9 -> n1
moving n1 -> n2
moving n2 -> n10


True

In [102]:
#print rect_graph.data

In [ ]:
path=rect_graph.get_path("n3","n2")

In [ ]:
rect_graph.move_to(path)

In [101]:
rect_graph.move_to_node("n1")

moving n6 -> n1


In [9]:
xml=AsciiDataTable_to_XMLDataTable(table)

In [ ]:
xml.to_tuple_list(xml.get_attribute_names())

In [ ]:
xml.to_list("a")

In [10]:
xml.attribute_names

['a', 'b', 'c']

In [11]:
xml.data

[['1', '2', '3'], ['4', '5', '6']]

In [103]:
rect_graph.state

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [64]:
path=rect_graph.get_path("n5","n1")

In [65]:
rect_graph.path_length(path)

0.051000000000000004

In [176]:
rect_graph.move_to_node("n1")

moving n1 -> n2
moving n2 -> n1


In [25]:
rect_graph.move_to_node("n1")
for node in rect_graph.node_names:
    path=rect_graph.get_path("n1",node)
    print("The length of the path n1-> {0} in s is {1}".format(node,rect_graph.path_length(path)))

moving n1 -> n2
moving n2 -> n1
The length of the path n1-> n1 in s is 0.018
The length of the path n1-> n2 in s is 0.012
The length of the path n1-> n3 in s is 0.012
The length of the path n1-> n4 in s is 0.057
The length of the path n1-> n5 in s is 0.163
The length of the path n1-> n6 in s is 0.113
The length of the path n1-> n7 in s is 0.006
Warning the path length is less than 1 microsecond,make sure num_repeats is high enough to measure it.
The length of the path n1-> n8 in s is 0.0
The length of the path n1-> n9 in s is 0.006
The length of the path n1-> n10 in s is 0.016


In [125]:
rect_graph.state

[1, 0, 0, 0, 0, 0]

In [126]:
rect_graph.move_to_node("n1")

moving n1 -> n2
moving n2 -> n1


In [90]:
html=rect_graph.data.to_html(index=False)

In [91]:
df=pandas.read_html(html)

In [24]:
rect_graph.edges

['edge_n1_n2_000',
 'edge_n2_n1_000',
 'edge_n3_n1_000',
 'edge_n1_n3_000',
 'edge_n4_n2_000',
 'edge_n2_n4_000',
 'edge_n5_n1_000',
 'edge_n1_n5_000',
 'edge_n6_n1_000',
 'edge_n1_n6_000',
 'edge_n7_n1_000',
 'edge_n1_n7_000',
 'edge_n8_n1_000',
 'edge_n1_n8_000',
 'edge_n9_n1_000',
 'edge_n1_n9_000',
 'edge_n10_n2_000',
 'edge_n2_n10_000',
 'edge_n7_n4_000']

In [111]:
table=OnePortCalrepModel(os.path.join(TESTS_DIRECTORY,'700437.asc'))
rect_graph.set_state("n2",table)

The self.metadata["Device_Id"] variable is 700437


In [116]:
rect_graph.move_to_node("n1")

moving n3 -> n1


In [114]:
print rect_graph.data

Test.hdf


In [8]:
from ipywidgets import *


In [23]:
def move_and_print(end_node_number):
    node=rect_graph.node_names[end_node_number]
    rect_graph.move_to_node(node)
    print("*"*80)
    print("{0}".format(rect_graph.node_descriptions[end_node_number])+" {0}".format(rect_graph.state))
    print("*"*80)
    print rect_graph.data
    
interact(move_and_print,end_node_number=(0,len(rect_graph.node_names)-1,1))

moving n3 -> n1
moving n1 -> n2
********************************************************************************
AsciiDataTable [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
********************************************************************************
Frequency,imS11,imS12,imS13,imS14,imS21,imS22,imS23,imS24,imS31,imS32,imS33,imS34,imS41,imS42,imS43,imS44,reS11,reS12,reS13,reS14,reS21,reS22,reS23,reS24,reS31,reS32,reS33,reS34,reS41,reS42,reS43,reS44
1.0,-0.008622,0,0.13116,0,0,0.011407,0,-0.183539,0.13116,0,0.041095,0,0,0.896714,0,-0.001502,-0.002222,0,0.952784,0,0,-0.015465,0,-0.949271,0.952784,0,0.004453,0,0,0.169317,0,0.002838
1.1,0.008366,0,-0.962076,0,0,0.03435,0,-0.387847,-0.962076,0,0.001709,0,0,-0.549641,0,0.025274,-0.001188,0,0.145232,0,0,-0.012758,0,0.876401,0.145232,0,0.001075,0,0,0.723761,0,0.034554
1.2,-0.00689,0,0.140074,0,0,0.063693,0,-0.810093,0.140074,0,-0.006566,0,0,0.386759,0,0.065747,0.003945,0,0.973279,0,0,0.008635,0,0.49787,0.973279,0,0.050593,0,0,0.825344,0,0.074783
1.3,0.00

In [140]:
json=data_frame.to_json("test.json",orient='records')

In [153]:
json_string=data_frame.to_json(orient='records')

In [155]:
new_df=pandas.read_json(json_string,orient='records')

In [141]:
new_df=pandas.read_json("test.json",orient='records')

In [152]:
#str(table.get_data_dictionary_list())
json

In [43]:
rect_graph.node_descriptions


['Pandas Data Frame',
 'AsciiDataTable',
 'HDF File',
 'XML Data Table',
 'Excel File',
 'HTML String',
 'JSON File',
 'JSON String',
 'CSV File',
 'Matlab File']

In [161]:
import json
test_list=json.load(open("test.json",'r'))

In [166]:
text_xml=DataTable(None,**{"data_dictionary":{"data":test_list}})

In [181]:
#print text_xml

In [182]:
rect_graph.edges

['edge_n1_n2_000',
 'edge_n2_n1_000',
 'edge_n3_n1_000',
 'edge_n1_n3_000',
 'edge_n4_n2_000',
 'edge_n2_n4_000',
 'edge_n5_n1_000',
 'edge_n1_n5_000',
 'edge_n6_n1_000',
 'edge_n1_n6_000',
 'edge_n7_n1_000',
 'edge_n1_n7_000',
 'edge_n8_n1_000',
 'edge_n1_n8_000',
 'edge_n7_n4_000']

In [184]:
rect_graph.move_to_node("n7")

moving n7 -> n1
moving n1 -> n7


In [185]:
rect_graph.move_to_node("n4")

moving n7 -> n4


In [189]:
#print rect_graph.data

In [10]:
from scipy.io import savemat,loadmat

In [11]:
savemat("data.mat",{"data":table.data,"column_names":table.column_names})

In [12]:
data=loadmat("data.mat")

In [13]:
new_table=AsciiDataTable(None,column_names=map(lambda x: x.rstrip().lstrip(),data["column_names"].tolist()),
                         data=data["data"].tolist())

In [26]:
rect_graph.move_to_node("n4")
xml=rect_graph.data

moving n10 -> n2
moving n2 -> n4


In [34]:
import sqlite3
import odo

In [39]:
data_frame.dtypes[0]

dtype('float64')

In [42]:
conn = sqlite3.connect("test.db")
conn.execute("create table data")
odo.odo(data_frame,'sqlite:///test.db::data')

OperationalError: near "data": syntax error